In [2]:
import os
from os import listdir
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as Layers
import tensorflow.keras.models as Models
from tensorflow.keras.metrics import categorical_crossentropy
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer

In [3]:
list(os.listdir("../input/osic-pulmonary-fibrosis-progression"))

In [4]:
trainDataFrame = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
testDataFrame = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')

In [5]:
print(trainDataFrame.shape)
print("---------------------------------------------------")
print(testDataFrame.shape)
print("---------------------------------------------------")
print(trainDataFrame)
print("---------------------------------------------------")
print(testDataFrame)

In [6]:
print(f"The total patient ids are {trainDataFrame['Patient'].count()}")
print(f"Number of unique ids are {trainDataFrame['Patient'].value_counts().shape[0]} ")

In [7]:
trainDataFrame.drop_duplicates(subset=['Patient','Weeks'], keep = False, inplace = True)

In [8]:

submissionDataFrame = pd.read_csv("../input/osic-pulmonary-fibrosis-progression/sample_submission.csv")
print(f"The sample submission contains: {submissionDataFrame.shape[0]} rows and {submissionDataFrame.shape[1]} columns.")
submissionDataFrame[['Patient','Weeks']] = submissionDataFrame.Patient_Week.str.split("_",expand = True)
submissionDataFrame =  submissionDataFrame[['Patient','Weeks','Confidence', 'Patient_Week']]
submissionDataFrame = submissionDataFrame.merge(testDataFrame.drop('Weeks', axis = 1), on = "Patient")

In [9]:

trainDataFrame['Source'] = 'train'
submissionDataFrame['Source'] = 'test'
dataFrame = trainDataFrame.append([submissionDataFrame])
dataFrame.reset_index(inplace = True)
dataFrame.head()

In [10]:
def get_week(df):
    
    copy = df.copy()
    copy['Weeks'] = copy['Weeks'].astype(int)
    copy.loc[copy.Source == 'test','min_week'] = np.nan
    copy["min_week"] = copy.groupby('Patient')['Weeks'].transform('min')
    copy['baselined_week'] = copy['Weeks'] - copy['min_week'] 
    return copy  

In [11]:
def get_base_FVC(df):
    # same as above
    copy = df.copy()
    base = copy.loc[copy.Weeks == copy.min_week]
    base = base[['Patient','FVC']].copy()
    base.columns = ['Patient','base_FVC']
    
    # add a row which contains the cumulated sum of rows for each patient
    base['nb'] = 1
    base['nb'] = base.groupby('Patient')['nb'].transform('cumsum')
    
    # drop all except the first row for each patient (=unique rows!), containing the min_week
    base = base[base.nb == 1]
    base.drop('nb', axis = 1, inplace = True)
    
    # merge the rows containing the base_FVC on the original _df
    copy = copy.merge(base, on = 'Patient', how = 'left')    
    copy.drop(['min_week'], axis = 1)
    
    return copy

In [12]:
dataFrame = get_week(dataFrame)
dataFrame = get_base_FVC(dataFrame)
dataFrame.drop(0)
print(dataFrame.head())
not_to_transform_attribute = ['Patient', 'Weeks', 'min_week']
transform_Attributes = ['FVC', 'Percent', 'Age', 'baselined_week', 'base_FVC']
cF = ['Sex', 'SmokingStatus']

In [35]:
def own_MinMaxColumnScaler(dataFrame, columns):
    """Adds columns with scaled numeric values to range [0, 1]
    using the formula X_scld = (X - X.min) / (X.max - X.min)"""
    for col in columns:
        new_col_name = col + '_scld'
        col_min = dataFrame[col].min()
        col_max = dataFrame[col].max()        
        dataFrame[new_col_name] = (dataFrame[col] - col_min) / ( col_max - col_min )
def own_OneHotColumnCreator(dataFrame, columns):
    """OneHot Encodes categorical features. Adds a column for each unique value per column"""
    for col in cF:
        for value in dataFrame[col].unique():
            dataFrame[value] = (dataFrame[col] == value).astype(int)

In [36]:
own_MinMaxColumnScaler(dataFrame, transform_Attributes)

own_OneHotColumnCreator(dataFrame, cF)

In [37]:
#print(dataFrame)
dataFrame[dataFrame.Source != "train"].head()

In [38]:
train_df = dataFrame.loc[dataFrame.Source == 'train']
sub = dataFrame.loc[dataFrame.Source == 'test']



In [39]:
featureList = ['baselined_week_scld', 'Percent_scld', 'Age_scld', 'base_FVC_scld', 'Male', 'Female', 'Ex-smoker', 'Never smoked', 'Currently smokes']
EPOCHS = 100
BATCH_SIZE = 200
_lambda = 0.8
## Optimizers
ADAM = tf.keras.optimizers.Adam(lr = 0.1,beta_1 = 0.9, beta_2 = 0.999,decay = 0.01)
SGD = tf.keras.optimizers.SGD()
optimizer = ADAM

In [40]:
C1, C2 = tf.constant(70, dtype='float32'), tf.constant(1000, dtype="float32")
def score(y_true, y_pred):
    """Calculate the competition metric"""
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    sigma = y_pred[:, 2] - y_pred[:, 0]
    fvc_pred = y_pred[:, 1]
    sigma_clip = tf.maximum(sigma, C1)
    delta = tf.abs(y_true[:, 0] - fvc_pred)
    delta = tf.minimum(delta, C2)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype = tf.float32) )
    metric = (delta / sigma_clip) * sq2 + tf.math.log(sigma_clip * sq2)
    return K.mean(metric)
def qloss(y_true, y_pred):
    """Calculate Pinball loss"""
    # IMPORTANT: define quartiles, feel free to change here!
    qs = [0.2, 0.50, 0.8]
    q = tf.constant(np.array([qs]), dtype = tf.float32)
    e = y_true - y_pred
    v = tf.maximum(q * e, (q-1) * e)
    return K.mean(v)
#pinball loss function
def mloss(_lambda):
    """Combine Score and qloss"""
    def loss(y_true, y_pred):
        print("lol1")
        print(y_true, y_pred)
        return _lambda * qloss(y_true, y_pred) + (1 - _lambda) * score(y_true, y_pred)
    return loss

In [41]:
def get_model():
    inp = Layers.Input((len(featureList),), name = "Patient")
    x = Layers.Dense(128, activation = "relu", name = "d1")(inp)
    x = Layers.Dropout(0.25)(x)
    x = Layers.Dense(128, activation = "relu", name = "d2")(x)
    x = Layers.Dropout(0.2)(x)
    # predicting the 
    p1 = Layers.Dense(3, activation = "relu", name = "p1")(x)
    # quantile adjusting p1 predictions
    p2 = Layers.Dense(3, activation = "relu", name = "p2")(x)
    preds = Layers.Lambda(lambda x: x[0] + tf.cumsum(x[1], axis = 1), 
                     name = "preds")([p1, p2])
    print("lol")
    print(_lambda)
    model = Models.Model(inp, preds, name = "NeuralNet")
    model.compile(loss = mloss(_lambda), optimizer = optimizer, metrics = [score])
    
    return model

In [42]:
neuralNet = get_model()
neuralNet.summary()

In [71]:

y = trainDataFrame['FVC'].values
y=y.astype(float)
# get training & test data
X_train = train_df[featureList].values
X_test = sub[featureList].values

# instantiate target arrays
train_preds = np.zeros((X_train.shape[0], 3))
test_preds = np.zeros((X_test.shape[0], 3))


In [72]:
NFOLD = 5
kf = KFold(n_splits = NFOLD)
count = 0
for train_idx, val_idx in kf.split(X_train):
    count += 1
    print(f"FOLD {count}:")
    net = get_model()
    net.fit(X_train[train_idx], y[train_idx], batch_size = BATCH_SIZE, epochs = EPOCHS, verbose = 1 ,validation_data = (X_train[val_idx], y[val_idx]), shuffle = True , workers = 3) 
    print("Train:", net.evaluate(X_train[train_idx], y[train_idx], verbose = 1, batch_size = BATCH_SIZE))
    print("Val:", net.evaluate(X_train[val_idx], y[val_idx], verbose = 0, batch_size = BATCH_SIZE))
    train_preds[val_idx] = net.predict(X_train[val_idx], batch_size = BATCH_SIZE, verbose = 0)
    print("Predicting Test...")
    test_preds += net.predict(X_test, batch_size = BATCH_SIZE, verbose = 0) / NFOLD

In [73]:
sigma_opt = mean_absolute_error(y, train_preds[:,1])
sigma_uncertain = train_preds[:,2] - train_preds[:,0]
sigma_mean = np.mean(sigma_uncertain)
print(sigma_opt, sigma_mean)

In [74]:
Submission.head()

In [ ]:
Submission['FVC1'] = test_preds[:, 1]
Submission['Confidence1'] = test_preds[:,2] - test_preds[:,0]
submission = Submission[['Patient_Week','FVC','Confidence','FVC1','Confidence1']].copy()
submission.loc[~submission.FVC1.isnull()].head(10)

In [ ]:
submission.loc[~submission.FVC1.isnull(),'FVC'] = submission.loc[~submission.FVC1.isnull(),'FVC1']

if sigma_mean < 70:
    submission['Confidence'] = sigma_opt
else:
    submission.loc[~submission.FVC1.isnull(),'Confidence'] = submission.loc[~submission.FVC1.isnull(),'Confidence1']

In [ ]:
submission.head()

In [ ]:
submission.describe().T

In [ ]:
org_test = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')
for i in range(len(org_test)):
    submission.loc[submission['Patient_Week']==org_test.Patient[i]+'_'+str(org_test.Weeks[i]), 'FVC'] = org_test.FVC[i]
    submission.loc[submission['Patient_Week']==org_test.Patient[i]+'_'+str(org_test.Weeks[i]), 'Confidence'] = 70

In [ ]:
submission[["Patient_Week","FVC","Confidence"]].to_csv("/kaggle/working/submission.csv", index = False)